In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# Vanilla LSTM

In [2]:
model = tf.keras.models.Sequential()

In [3]:
model.add(layers.LSTM(20, input_shape=(10, 1),return_sequences=True))

In [4]:
model.add(layers.TimeDistributed(layers.Dense(1, activation="sigmoid")))

In [44]:
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["acc"]
)

In [45]:
import numpy as np

In [46]:
n_timestamps = 10

In [47]:
X = np.random.rand(n_timestamps)

In [48]:
limit = n_timestamps / 4.0

In [49]:
y = (np.cumsum(X) > limit).astype(np.int32) 

In [50]:
y

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=int32)

In [51]:
X = np.expand_dims(np.expand_dims(X, -1), 0)
y = np.expand_dims(np.expand_dims(y, -1), 0)

In [52]:
X.shape

(1, 10, 1)

In [53]:
y.shape

(1, 10, 1)

In [54]:
model.fit(X, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 793ms/step - loss: 0.6821 - acc: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6820 - acc: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6819 - acc: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6818 - acc: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6817 - acc: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6816 - acc: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6816 - acc: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6815 - acc: 0.5000
Epoch 9/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6814 - acc: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6813 - acc: 0.5000


In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 20)            1760      
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


# BiLSTM

In [56]:
bilstm_model = tf.keras.models.Sequential()

In [57]:
bilstm_model.add(layers.Bidirectional(layers.LSTM(20, return_sequences=True), input_shape=(n_timestamps, 1)))
bilstm_model.add(layers.TimeDistributed(layers.Dense(1, activation="sigmoid")))

In [58]:
bilstm_model.compile(
    optimizer=tf.train.AdadeltaOptimizer(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["acc"]
)

In [59]:
bilstm_model.fit(X, y, epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6902 - acc: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6902 - acc: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6902 - acc: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6902 - acc: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6902 - acc: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6902 - acc: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6901 - acc: 0.5000
Epoch 8/10
1/1 [==============================] - 0s 18ms/step - loss: 0.6901 - acc: 0.5000
Epoch 9/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6901 - acc: 0.5000
Epoch 10/10
1/1 [==============================] - 0s 20ms/step - loss: 0.6901 - acc: 0.5000


In [61]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 20)            1760      
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             21        
Total params: 1,781
Trainable params: 1,781
Non-trainable params: 0
_________________________________________________________________


In [62]:
bilstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 10, 40)            3520      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             41        
Total params: 3,561
Trainable params: 3,561
Non-trainable params: 0
_________________________________________________________________


# BLSTM for frames

In [64]:
frame_dim = 1024
player_dim = 3472

In [63]:
num_hidden_states = 256
embedding_dim = 256

In [73]:
n_timestamps = 24

In [76]:
model = tf.keras.models.Sequential()
model.add(layers.Embedding(frame_dim, embedding_dim, input_length=n_timestamps))
model.add(layers.Bidirectional(layers.LSTM(num_hidden_states, return_sequences=True), input_shape=(n_timestamps, embedding_dim)))

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 24, 256)           262144    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 24, 512)           1050624   
Total params: 1,312,768
Trainable params: 1,312,768
Non-trainable params: 0
_________________________________________________________________


# Classification Network

In [78]:
import tensorflow.keras.backend as K

In [308]:
batch_size=5

In [149]:
k = 11

In [309]:
ft = layers.Input(shape=(n_timestamps, frame_dim))
pti = layers.Input(shape=(n_timestamps, None, player_dim))

ft_embed = layers.Dense(embedding_dim, activation="relu")(ft)
pti_embed = layers.Dense(embedding_dim, activation="relu")(pti)

h_f = layers.Bidirectional(layers.LSTM(num_hidden_states, return_sequences=True), input_shape=(n_timestamps, embedding_dim))(ft_embed)

def n_players_permute(tensor):
    num_players = K.shape(tensor)[2]
    return num_players, K.permute_dimensions(tensor, [0, 2, 1, 3])

def reshape1(tensor):
    return K.reshape(tensor, shape=(-1, n_timestamps, embedding_dim))

def reshape2(tensor):
    return K.reshape(tensor, shape=(-1, num_players, n_timestamps, 2 * num_hidden_states))

def permute(tensor):
    return K.permute_dimensions(tensor, [0, 2, 1, 3])


#pti_embed = K.permute_dimensions(pti_embed, [0, 2, 1, 3])
num_players, pti_embed = layers.Lambda(n_players_permute)(pti_embed)
pti_embed = layers.Lambda(reshape1)(pti_embed)

h_pi = layers.Bidirectional(layers.LSTM(num_hidden_states, return_sequences=True), input_shape=(n_timestamps, embedding_dim))(pti_embed)
h_pi = layers.Lambda(reshape2)(h_pi)
#h_pi = layers.Reshape(target_shape=(num_players, n_timestamps, num_hidden_states))(h_pi)
h_pi = layers.Lambda(permute)(h_pi)

def attention(tensor):
    return K.mean(tensor, axis=2)

a_t = layers.Lambda(attention)(h_pi)

def concat(args):
    return K.concatenate(args, axis=-1)

c_t = layers.Lambda(concat)([h_f, a_t])
h_e = layers.LSTM(num_hidden_states, return_sequences=True, input_shape=(n_timestamps, 2 * embedding_dim))(c_t)
outputs = layers.Dense(k, activation="linear")(h_e)


In [310]:
a_t

<tf.Tensor 'lambda_63/Mean:0' shape=(?, 24, 512) dtype=float32>

In [387]:
outputs

<tf.Tensor 'dense_97/BiasAdd:0' shape=(?, 24, 11) dtype=float32>

In [397]:
def squared_hinge_loss(y_true, y_pred):
    # y_true 100x11, y_pred 100x24x11
    # print(y_pred.get_shape())
    L = 0.5 * K.sum(K.square(K.maximum(0., 1. - y_true * y_pred)), axis=[-1, -2])
    return K.mean(L)

In [398]:
class_model.compile(
    optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
    loss=squared_hinge_loss
)

## Mock Data

In [354]:
class_model.summary(line_length=150)

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_81 (InputLayer)                            (None, 24, None, 3472)           0                                                                   
______________________________________________________________________________________________________________________________________________________
dense_96 (Dense)                                 (None, 24, None, 256)            889088            input_81[0][0]                                    
______________________________________________________________________________________________________________________________________________________
lambda_59 (Lambda)                               [(), (None, None, 24, 256)]      0           

In [329]:
N_SAMPLES = 100
N_CLASS = 11

In [303]:
np.random.randint(6, 11)

6

In [288]:
frame = np.random.rand(N_SAMPLES, n_timestamps, frame_dim)
player = np.random.rand(N_SAMPLES, n_timestamps, np.random.randint(6, 11), player_dim)

In [337]:
labels = -1 * np.ones(shape=(N_SAMPLES, N_CLASS))
classes = np.random.randint(0, N_CLASS, N_SAMPLES)

In [338]:
classes.shape

(100,)

In [339]:
labels[np.arange(len(labels)), classes] = 1

In [347]:
labels[70]

array([-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [402]:
np.(labels, axis=0, repeats=24).shape

(2400, 11)

In [403]:
labels = labels.reshape(labels.shape[0], 1, labels.shape[1])

In [404]:
labels.shape

(100, 1, 11)

In [406]:
labels = labels.repeat(24, axis=1)

In [356]:
labels.shape

(100, 11)

In [306]:
player.shape

(100, 24, 7, 3472)

In [407]:
class_model.fit([frame, player], labels, epochs=5, batch_size=100)

Epoch 1/5
100/100 [==============================] - 21s 212ms/step - loss: 121.6298
Epoch 2/5
100/100 [==============================] - 4s 37ms/step - loss: 268.4391
Epoch 3/5
100/100 [==============================] - 4s 36ms/step - loss: 183.1513
Epoch 4/5
100/100 [==============================] - 4s 38ms/step - loss: 110.3560
Epoch 5/5
100/100 [==============================] - 4s 37ms/step - loss: 63.5767
